## 1) Optuna For Hyper Parameter Tuning Using Baysian Search
Trying to estimate a function f(hyperparmeters) = accuracy (metric under consideration) by getting values on some hyper parameters and when it able to visualize f(hyperparameters) , them find the point that maximize this function , and give the Corresponding Hyper Parameter value.

### Key Terms for Optuna
Before diving into Optuna for hyperparameter tuning, it's helpful to understand some key terminologies that are commonly used within the framework. Here's a glossary of important terms:
1. Study
• A study in Optuna is an optimization session that encompasses multiple trials. It's essentially a collection of trials aimed at optimizing the objective function. You can think of a study as the overall experiment or search process.
• Example: A study to find the best hyperparameters for an XGBoost model.

2. Trial
• A trial is a single iteration of the optimization process where a specific set of hyperparameters is evaluated. Each trial runs the objective function once with a distinct set of hyperparameters.
• Example: One trial could involve training a model with a learning rate of 0.01 and a max depth of 5.

3. Trial Parameters
• These are the specific hyperparameter values chosen during a trial. Each trial will have a unique combination of hyperparameters that are evaluated to see how they impact the objective function.
• Example: In one trial, the learning rate might be 0.001, while the batch size is 32, and in another trial, the learning rate could be 0.01 with a batch size of 64.

4. Objective Function
• The objective function is the function to be optimized (minimized or maximized) during the hyperparameter search. It takes hyperparameters as input and returns a value (such as accuracy, loss, or any other metric) that Optuna tries to optimize.
• Example: In a classification task, the objective function could be the cross-entropy loss, which Optuna seeks to minimize.

5. Sampler
• A sampler is the algorithm that suggests which hyperparameters should be evaluated next. Optuna uses the Tree-structured Parzen Estimator (TPE) by default, but it also supports other sampling methods like Random Search or even custom samplers.
• Example: TPE suggests promising areas of the hyperparameter space, focusing on regions that are likely to yield better results.


#### Lets use Optuna


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 5.9 MB/s eta 0:00:006.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.1 MB/s eta 0:00:0031m45.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [32]:
import optuna
from sklearn.model_selection import train_test_split
# from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

In [33]:
# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [35]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [36]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [37]:
## No null values

In [38]:
# Split into Features X and y
X = df.drop('Outcome',axis=1)
y = df['Outcome']

# split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
# Optional: Scale the data for better model performance (aS We don't neeed Scalerization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape


((537, 8), (231, 8))

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Define Objective

def obective(trial):

    ## Defining Range or possible value of Hyper parameters
    n_estimators = trial.suggest_int('n_estimators',50,200)
    max_depth = trial.suggest_int('max_depth',3,20)

    ## create the RandomForesClassifier with suggest hyperparameters
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   random_state=42)
    
    ## perform 3-fold cross-validation and Calculate Accuracy
    score = cross_val_score(model, X_train,y_train,cv=3, scoring='accuracy').mean()

    return score



### Creating Study and optimize

In [55]:
study= optuna.create_study(direction = "maximize",sampler = optuna.samplers.TPESampler())
study.optimize(obective, n_trials = 50)

[I 2025-07-12 18:26:16,330] A new study created in memory with name: no-name-ea44f54a-f3e4-4bb3-a497-222bf7a855f9
[I 2025-07-12 18:26:16,505] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 91, 'max_depth': 19}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-12 18:26:16,729] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 163, 'max_depth': 4}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-12 18:26:16,943] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 149, 'max_depth': 5}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-12 18:26:17,159] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 136, 'max_depth': 11}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-12 18:26:17,258] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 61, 'max_depth': 15}. Best is trial 0 with value: 0.77467411

In [56]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7951582867783985
Best hyperparameters: {'n_estimators': 95, 'max_depth': 8}


In [57]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


#### Using grid Search for Sampler

##### Random Sampler 

In [45]:
study = optuna.create_study(direction='maximize',sampler= optuna.samplers.RandomSampler())
study.optimize(obective, n_trials=50)

[I 2025-07-12 18:17:26,687] A new study created in memory with name: no-name-c555cc4d-258b-4ebd-8295-83ad9b39c17b
[I 2025-07-12 18:17:27,054] Trial 0 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 193, 'max_depth': 17}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-07-12 18:17:27,277] Trial 1 finished with value: 0.7914338919925511 and parameters: {'n_estimators': 145, 'max_depth': 8}. Best is trial 1 with value: 0.7914338919925511.
[I 2025-07-12 18:17:27,403] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 86, 'max_depth': 5}. Best is trial 1 with value: 0.7914338919925511.
[I 2025-07-12 18:17:27,506] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 64, 'max_depth': 16}. Best is trial 1 with value: 0.7914338919925511.
[I 2025-07-12 18:17:27,646] Trial 4 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 89, 'max_depth': 9}. Best is trial 1 with value: 0.7914338919

In [46]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7914338919925511
Best hyperparameters: {'n_estimators': 145, 'max_depth': 8}


In [47]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


##### Grid Search , By Defining Our custom search

In [49]:
search_space = {
    'n_estimators' : [50,100,150,200],
    'max_depth' : [5,10,15,20]
}

In [51]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(obective)

[I 2025-07-12 18:23:45,313] A new study created in memory with name: no-name-042dc1e2-ca02-4f08-9e47-166443e5d98b
[I 2025-07-12 18:23:45,482] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-12 18:23:45,716] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-07-12 18:23:45,803] Trial 2 finished with value: 0.7802607076350094 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7802607076350094.
[I 2025-07-12 18:23:45,965] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7802607076350094.
[I 2025-07-12 18:23:46,139] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.780260

In [52]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 50, 'max_depth': 15}


In [53]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## 2) Visualization in Optuna

1) Here I have re do study using TPE sample with 50 trial.   
2) And want to visualize it.  

In [59]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [60]:
plot_optimization_history(study).show()

In [61]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [62]:
# 3. Slice Plot
plot_slice(study).show()

In [63]:
# 4. Contour Plot
plot_contour(study).show()

In [64]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Define by Search Run (optimizing Multiple ML Models)
  Not Only Search Space optimization Also give which ML algo is best

In [65]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [66]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [67]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100)

[I 2025-07-12 18:43:29,824] A new study created in memory with name: no-name-774368fd-ccd6-41fa-b0f9-9481791c105c
[I 2025-07-12 18:43:31,012] Trial 0 finished with value: 0.7541899441340782 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 270, 'learning_rate': 0.13261984643853306, 'max_depth': 18, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.7541899441340782.
[I 2025-07-12 18:43:31,370] Trial 1 finished with value: 0.7523277467411544 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 109, 'learning_rate': 0.14477330637411032, 'max_depth': 7, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7541899441340782.
[I 2025-07-12 18:43:31,418] Trial 2 finished with value: 0.7299813780260708 and parameters: {'classifier': 'SVM', 'C': 67.5711841056782, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.7541899441340782.
[I 2025-07-12 18:43:31,672] Trial 3 finished with value: 0.769087523

In [68]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'RandomForest', 'n_estimators': 154, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 4, 'bootstrap': False}
Best trial accuracy: 0.7895716945996275


In [69]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.754190,2025-07-12 18:43:29.832342,2025-07-12 18:43:31.012626,0 days 00:00:01.180284,NaN,NaN,GradientBoosting,NaN,NaN,0.132620,18.0,5.0,7.0,270.0,COMPLETE
1,1,0.752328,2025-07-12 18:43:31.013016,2025-07-12 18:43:31.370462,0 days 00:00:00.357446,NaN,NaN,GradientBoosting,NaN,NaN,0.144773,7.0,2.0,6.0,109.0,COMPLETE
2,2,0.729981,2025-07-12 18:43:31.370818,2025-07-12 18:43:31.418203,0 days 00:00:00.047385,67.571184,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.769088,2025-07-12 18:43:31.418666,2025-07-12 18:43:31.672679,0 days 00:00:00.254013,NaN,NaN,GradientBoosting,NaN,NaN,0.047573,15.0,9.0,3.0,80.0,COMPLETE
4,4,0.778399,2025-07-12 18:43:31.673031,2025-07-12 18:43:32.007146,0 days 00:00:00.334115,NaN,False,RandomForest,NaN,NaN,NaN,19.0,1.0,3.0,214.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.782123,2025-07-12 18:43:55.031401,2025-07-12 18:43:55.281169,0 days 00:00:00.249768,NaN,False,RandomForest,NaN,NaN,NaN,15.0,3.0,3.0,161.0,COMPLETE
96,96,0.787709,2025-07-12 18:43:55.281532,2025-07-12 18:43:55.533040,0 days 00:00:00.251508,NaN,False,RandomForest,NaN,NaN,NaN,18.0,4.0,3.0,172.0,COMPLETE
97,97,0.785847,2025-07-12 18:43:55.533444,2025-07-12 18:43:55.740342,0 days 00:00:00.206898,NaN,False,RandomForest,NaN,NaN,NaN,17.0,4.0,2.0,141.0,COMPLETE
98,98,0.780261,2025-07-12 18:43:55.740675,2025-07-12 18:43:55.991545,0 days 00:00:00.250870,NaN,False,RandomForest,NaN,NaN,NaN,15.0,5.0,8.0,179.0,COMPLETE


In [70]:
study.trials_dataframe()['params_classifier'].value_counts()

RandomForest        79
GradientBoosting    11
SVM                 10
Name: params_classifier, dtype: int64

In [71]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.747249
RandomForest        0.779106
SVM                 0.738547
Name: value, dtype: float64

In [72]:
# 1. Optimization History
plot_optimization_history(study).show()

In [73]:
# 3. Slice Plot
plot_slice(study).show()

In [74]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

### Using Xgboost

In [82]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-07-12 18:50:40,546] A new study created in memory with name: no-name-177fe3c2-2d50-4e1c-b648-9b35fba055c0


[0]	train-mlogloss:0.91700	eval-mlogloss:0.90768
[1]	train-mlogloss:0.82308	eval-mlogloss:0.80694
[2]	train-mlogloss:0.73972	eval-mlogloss:0.72557
[3]	train-mlogloss:0.72665	eval-mlogloss:0.71288
[4]	train-mlogloss:0.63780	eval-mlogloss:0.62277
[5]	train-mlogloss:0.62565	eval-mlogloss:0.61154
[6]	train-mlogloss:0.57853	eval-mlogloss:0.56618
[7]	train-mlogloss:0.57362	eval-mlogloss:0.55902
[8]	train-mlogloss:0.57341	eval-mlogloss:0.56004
[9]	train-mlogloss:0.56616	eval-mlogloss:0.54550
[10]	train-mlogloss:0.51907	eval-mlogloss:0.48846
[11]	train-mlogloss:0.51651	eval-mlogloss:0.48099
[12]	train-mlogloss:0.51574	eval-mlogloss:0.48021
[13]	train-mlogloss:0.51270	eval-mlogloss:0.47766
[14]	train-mlogloss:0.51279	eval-mlogloss:0.48075
[15]	train-mlogloss:0.50977	eval-mlogloss:0.47512
[16]	train-mlogloss:0.50774	eval-mlogloss:0.47196
[17]	train-mlogloss:0.50322	eval-mlogloss:0.46714
[18]	train-mlogloss:0.49868	eval-mlogloss:0.46115
[19]	train-mlogloss:0.49761	eval-mlogloss:0.45928
[20]	train

[I 2025-07-12 18:50:40,842] Trial 0 finished with value: 0.9666666666666667 and parameters: {'lambda': 0.2723322301649356, 'alpha': 0.0006199378172146518, 'eta': 0.29942718215873765, 'gamma': 0.02155291074061066, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.46042758221754804, 'colsample_bytree': 0.6535633097191563}. Best is trial 0 with value: 0.9666666666666667.


[0]	train-mlogloss:0.98795	eval-mlogloss:0.97235
[1]	train-mlogloss:0.86120	eval-mlogloss:0.83676
[2]	train-mlogloss:0.77275	eval-mlogloss:0.74041
[3]	train-mlogloss:0.71706	eval-mlogloss:0.67689
[4]	train-mlogloss:0.65093	eval-mlogloss:0.61040
[5]	train-mlogloss:0.56433	eval-mlogloss:0.51899
[6]	train-mlogloss:0.52794	eval-mlogloss:0.47821
[7]	train-mlogloss:0.50228	eval-mlogloss:0.45105
[8]	train-mlogloss:0.46058	eval-mlogloss:0.40854
[9]	train-mlogloss:0.41356	eval-mlogloss:0.35965
[10]	train-mlogloss:0.38127	eval-mlogloss:0.32684
[11]	train-mlogloss:0.35976	eval-mlogloss:0.30242
[12]	train-mlogloss:0.34207	eval-mlogloss:0.28461
[13]	train-mlogloss:0.30919	eval-mlogloss:0.24956
[14]	train-mlogloss:0.29166	eval-mlogloss:0.23113
[15]	train-mlogloss:0.28762	eval-mlogloss:0.22782
[16]	train-mlogloss:0.27403	eval-mlogloss:0.21111
[17]	train-mlogloss:0.26121	eval-mlogloss:0.19713
[18]	train-mlogloss:0.24869	eval-mlogloss:0.18341
[19]	train-mlogloss:0.24792	eval-mlogloss:0.18357
[20]	train

[I 2025-07-12 18:50:40,929] Trial 1 finished with value: 1.0 and parameters: {'lambda': 5.719152467644261e-06, 'alpha': 0.0011574754659807764, 'eta': 0.14395867233218185, 'gamma': 2.392115282382619e-08, 'max_depth': 4, 'min_child_weight': 6, 'subsample': 0.6440016615454975, 'colsample_bytree': 0.44080715213184124}. Best is trial 1 with value: 1.0.


...

[0]	train-mlogloss:0.85501	eval-mlogloss:0.83943


[I 2025-07-12 18:50:44,107] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:0.90672	eval-mlogloss:0.89951


[I 2025-07-12 18:50:44,129] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 5.719152467644261e-06, 'alpha': 0.0011574754659807764, 'eta': 0.14395867233218185, 'gamma': 2.392115282382619e-08, 'max_depth': 4, 'min_child_weight': 6, 'subsample': 0.6440016615454975, 'colsample_bytree': 0.44080715213184124}
Best accuracy: 1.0


In [83]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()